In [95]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

today = datetime.today().strftime('%Y-%m-%d')
start_date = '2019-01-01'

eth_df = yf.download('BTC-USD',start_date, today)

eth_df.tail()

In [96]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-10-01'

eth_df = yf.download('ETH-USD',start_date, today)

eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-10,"$3,157.57","$3,177.21","$2,947.68","$3,083.10","$3,083.10",19535744145
2022-01-11,"$3,082.99","$3,247.51","$3,061.14","$3,238.11","$3,238.11",15211447193
2022-01-12,"$3,238.45","$3,401.22","$3,216.72","$3,372.26","$3,372.26",16662871689
2022-01-13,"$3,372.10","$3,396.97","$3,247.92","$3,248.29","$3,248.29",15294466275
2022-01-14,"$3,248.65","$3,330.77","$3,203.82","$3,310.00","$3,310.00",13562957230


In [97]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1528 entries, 2017-11-09 to 2022-01-14
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1528 non-null   float64
 1   High       1528 non-null   float64
 2   Low        1528 non-null   float64
 3   Close      1528 non-null   float64
 4   Adj Close  1528 non-null   float64
 5   Volume     1528 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 83.6 KB


In [98]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [99]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [100]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [101]:
df = eth_df[["Date", "Open"]]

new_names = {
    "Date": "ds", 
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [102]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [103]:
m = Prophet(
    seasonality_mode="multiplicative" 
)
m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -51.2144


    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        4397.8    0.00383495        888.06      0.2532           1      123   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       4433.78    0.00483577       179.563           1           1      240   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       4459.32    0.00668028       295.296           1           1      358   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       4477.86    0.00205316       202.327           1           1      480   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       4482.74    0.00139288       142.125           1           1      609   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     599     

In [104]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
1888,2023-01-10
1889,2023-01-11
1890,2023-01-12
1891,2023-01-13
1892,2023-01-14


In [105]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1888,2023-01-10,"$6,144.61","$5,149.29","$7,222.63"
1889,2023-01-11,"$6,026.23","$5,027.16","$7,102.40"
1890,2023-01-12,"$5,998.16","$5,015.47","$7,068.27"
1891,2023-01-13,"$5,992.86","$5,060.29","$7,083.14"
1892,2023-01-14,"$6,056.69","$5,094.59","$7,130.57"


In [106]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

3762.157075497809

In [107]:
plot_plotly(m, forecast)

In [108]:
plot_components_plotly(m, forecast)